In [1]:
# Import libraries

import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import glob
import os
import datetime
# from datetime import timedelta

# Import plotly express for EF plot
# import plotly.express as px
# import plotly.graph_objects as go
# px.defaults.width, px.defaults.height = 1000,600
# Set precision
pd.set_option('display.precision', 4)

In [2]:
df_option = pd.read_csv('options_final_total_1.csv', index_col=0)

In [3]:
# df_option.to_csv('colab_option.csv')

In [4]:
# Convert 'quote_datetime' to datetime
df_option['quote_datetime'] = pd.to_datetime(df_option['quote_datetime'])

In [5]:
df_option['date'] = df_option['quote_datetime'].dt.date
df_option['time'] = df_option['quote_datetime'].dt.time

In [6]:
df_option = df_option[df_option['quote_datetime'].dt.date != pd.to_datetime('2020-03-09').date()]

In [7]:
df_option = df_option[df_option['quote_datetime'].dt.date != pd.to_datetime('2020-11-09').date()]

In [8]:
# Calculate the number of unique days
num_unique_days = df_option['quote_datetime'].dt.date.nunique()

# Display the result
print(num_unique_days)

1320


In [9]:
df_option.columns

Index(['quote_datetime', 'strike', 'ask_C', 'ask_P', 'bid_C', 'bid_P', 'Open',
       'High', 'Low', 'Close', 'day_close', 'prev_close', 'gap',
       'prev_day_neg', 'range', 'realized_volatility_post30min',
       'realized_volatility_full_day', 'sma_50', 'sma_21', 'below_sma_50',
       'intraday_sma_21', 'gap931', 'date1', 'high931', 'low931', 'close931',
       'open931', 'straddle_exit', 'call_price', 'put_price', 'strike932',
       'date', 'time', 'call932', 'put932', 'straddle932', 'spot932'],
      dtype='object')

In [10]:
# Calculate rolling mean (SMA) and standard deviation for each date
df_option['Rolling_MA'] = (
    df_option.groupby(df_option['quote_datetime'].dt.date)['Close']
    .transform(lambda x: x.rolling(window=20, min_periods=1).mean())
)

df_option['Rolling_STD'] = (
    df_option.groupby(df_option['quote_datetime'].dt.date)['Close']
    .transform(lambda x: x.rolling(window=20, min_periods=1).std())
)

# Calculate Upper and Lower Bollinger Bands
df_option['Upper_Band'] = df_option['Rolling_MA'] + 2 * df_option['Rolling_STD']
df_option['Lower_Band'] = df_option['Rolling_MA'] - 1.5 * df_option['Rolling_STD']

In [11]:
# Filter rows where the time falls between 9:35 AM and 4:10 PM
start_time = pd.to_datetime('09:32:00').time()
end_time = pd.to_datetime('16:00:00').time()

# Apply the filter based on the time component of 'quote_datetime'
df3 = df_option[(df_option['quote_datetime'].dt.time >= start_time) & (df_option['quote_datetime'].dt.time <= end_time)]

In [12]:
df3.head()

,quote_datetime,strike,ask_C,ask_P,bid_C,bid_P,Open,High,Low,Close,...,date,time,call932,put932,straddle932,spot932,Rolling_MA,Rolling_STD,Upper_Band,Lower_Band
0,2018-01-02 09:32:00,2685.0,3.30,4.3,3.00,3.9,2685.60,2685.60,2684.30,2684.30,...,2018-01-02,09:32:00,3.15,4.1,7.25,2684.3,2684.3000,NaN,NaN,NaN
1,2018-01-02 09:33:00,2685.0,3.90,3.9,3.40,3.4,2684.32,2685.74,2684.23,2685.30,...,2018-01-02,09:33:00,3.15,4.1,7.25,2684.3,2684.8000,0.7071,2686.2142,2683.7393
2,2018-01-02 09:34:00,2685.0,3.10,4.5,2.70,4.2,2685.18,2685.18,2683.73,2683.73,...,2018-01-02,09:34:00,3.15,4.1,7.25,2684.3,2684.4433,0.7948,2686.0328,2683.2512
3,2018-01-02 09:35:00,2685.0,2.75,5.0,2.45,4.5,2683.80,2684.18,2683.05,2683.07,...,2018-01-02,09:35:00,3.15,4.1,7.25,2684.3,2684.1000,0.9448,2685.9896,2682.6828
4,2018-01-02 09:36:00,2685.0,2.65,5.0,2.55,4.7,2683.06,2683.23,2682.78,2682.94,...,2018-01-02,09:36:00,3.15,4.1,7.25,2684.3,2683.8680,0.9688,2685.8056,2682.4148


In [13]:
df3.columns

Index(['quote_datetime', 'strike', 'ask_C', 'ask_P', 'bid_C', 'bid_P', 'Open',
       'High', 'Low', 'Close', 'day_close', 'prev_close', 'gap',
       'prev_day_neg', 'range', 'realized_volatility_post30min',
       'realized_volatility_full_day', 'sma_50', 'sma_21', 'below_sma_50',
       'intraday_sma_21', 'gap931', 'date1', 'high931', 'low931', 'close931',
       'open931', 'straddle_exit', 'call_price', 'put_price', 'strike932',
       'date', 'time', 'call932', 'put932', 'straddle932', 'spot932',
       'Rolling_MA', 'Rolling_STD', 'Upper_Band', 'Lower_Band'],
      dtype='object')

In [14]:
df4 = df3.copy()

In [15]:
# Ensure the DataFrame is sorted by 'date' and 'time'
df4 = df4.sort_values(by=['date', 'time'])
df4['call_below_10per'] = False

# Define a function to filter each group based on the first occurrence of a condition
def filter_first_true(group):
    # Find the index of the first occurrence where 'call_price' <= 0.9 * 'call932'
    first_yes_idx = group[group['call_price'] <= 0.9 * group['call932']].index.min()
    
    # If such an index exists and is before 10:32 AM, process the group
    if not pd.isna(first_yes_idx):
        first_yes_time = group.loc[first_yes_idx, 'time']
        if first_yes_time < pd.to_datetime('10:32:00').time():
            group = group.loc[first_yes_idx:]
            group['call_below_10per'] = True
            return group
        
    # Otherwise, skip the group by returning an empty DataFrame
    return pd.DataFrame()

# Apply the filtering function to each group based on 'date'
df4 = df4.groupby('date').apply(filter_first_true).reset_index(drop=True)

In [16]:
df4.head()

,quote_datetime,strike,ask_C,ask_P,bid_C,bid_P,Open,High,Low,Close,...,time,call932,put932,straddle932,spot932,Rolling_MA,Rolling_STD,Upper_Band,Lower_Band,call_below_10per
0,2018-01-02 09:35:00,2685.0,2.75,5.0,2.45,4.5,2683.80,2684.18,2683.05,2683.07,...,09:35:00,3.15,4.1,7.25,2684.3,2684.1000,0.9448,2685.9896,2682.6828,True
1,2018-01-02 09:36:00,2685.0,2.65,5.0,2.55,4.7,2683.06,2683.23,2682.78,2682.94,...,09:36:00,3.15,4.1,7.25,2684.3,2683.8680,0.9688,2685.8056,2682.4148,True
2,2018-01-02 09:37:00,2685.0,2.60,5.3,2.45,4.7,2682.94,2683.22,2682.36,2682.80,...,09:37:00,3.15,4.1,7.25,2684.3,2683.6900,0.9700,2685.6301,2682.2350,True
3,2018-01-02 09:38:00,2685.0,3.30,4.3,3.10,3.8,2682.80,2684.01,2682.79,2683.90,...,09:38:00,3.15,4.1,7.25,2684.3,2683.7200,0.8891,2685.4981,2682.3864,True
4,2018-01-02 09:39:00,2685.0,3.80,3.6,3.40,3.2,2683.84,2685.09,2683.84,2685.07,...,09:39:00,3.15,4.1,7.25,2684.3,2683.8888,0.9515,2685.7917,2682.4615,True


In [17]:
# Ensure 'date' is a column or derived appropriately
df4['selling_time'] = (
    df4.groupby('date')['time']
    .transform('first')
)
df4['selling_time'] = df4['selling_time'].ffill()

In [18]:
df4.head()

,quote_datetime,strike,ask_C,ask_P,bid_C,bid_P,Open,High,Low,Close,...,call932,put932,straddle932,spot932,Rolling_MA,Rolling_STD,Upper_Band,Lower_Band,call_below_10per,selling_time
0,2018-01-02 09:35:00,2685.0,2.75,5.0,2.45,4.5,2683.80,2684.18,2683.05,2683.07,...,3.15,4.1,7.25,2684.3,2684.1000,0.9448,2685.9896,2682.6828,True,09:35:00
1,2018-01-02 09:36:00,2685.0,2.65,5.0,2.55,4.7,2683.06,2683.23,2682.78,2682.94,...,3.15,4.1,7.25,2684.3,2683.8680,0.9688,2685.8056,2682.4148,True,09:35:00
2,2018-01-02 09:37:00,2685.0,2.60,5.3,2.45,4.7,2682.94,2683.22,2682.36,2682.80,...,3.15,4.1,7.25,2684.3,2683.6900,0.9700,2685.6301,2682.2350,True,09:35:00
3,2018-01-02 09:38:00,2685.0,3.30,4.3,3.10,3.8,2682.80,2684.01,2682.79,2683.90,...,3.15,4.1,7.25,2684.3,2683.7200,0.8891,2685.4981,2682.3864,True,09:35:00
4,2018-01-02 09:39:00,2685.0,3.80,3.6,3.40,3.2,2683.84,2685.09,2683.84,2685.07,...,3.15,4.1,7.25,2684.3,2683.8888,0.9515,2685.7917,2682.4615,True,09:35:00


In [19]:
# Group by 'date' or another column if needed
df4['first_quote_datetime'] = df4.groupby('date')['quote_datetime'].transform('first')

# Filter rows where 'quote_datetime' is within 1 hour from the first 'quote_datetime' of the group
df_time = df4[df4['quote_datetime'] >= df4['first_quote_datetime'] + pd.Timedelta(hours=1)]

In [20]:
df_time

,quote_datetime,strike,ask_C,ask_P,bid_C,bid_P,Open,High,Low,Close,...,put932,straddle932,spot932,Rolling_MA,Rolling_STD,Upper_Band,Lower_Band,call_below_10per,selling_time,first_quote_datetime
60,2018-01-02 10:35:00,2685.0,7.90,0.95,7.40,0.90,2691.29,2691.73,2691.29,2691.71,...,4.10,7.25,2684.30,2690.8255,0.5783,2691.9821,2689.9581,True,09:35:00,2018-01-02 09:35:00
61,2018-01-02 10:36:00,2685.0,8.80,0.85,6.70,0.80,2691.72,2692.23,2691.72,2692.09,...,4.10,7.25,2684.30,2690.9150,0.6290,2692.1730,2689.9715,True,09:35:00,2018-01-02 09:35:00
62,2018-01-02 10:37:00,2685.0,8.80,0.80,7.00,0.75,2692.09,2692.39,2692.06,2692.33,...,4.10,7.25,2684.30,2691.0175,0.6846,2692.3868,2689.9906,True,09:35:00,2018-01-02 09:35:00
63,2018-01-02 10:38:00,2685.0,7.10,1.40,6.20,1.25,2692.35,2692.35,2690.16,2690.16,...,4.10,7.25,2684.30,2691.0160,0.6866,2692.3891,2689.9862,True,09:35:00,2018-01-02 09:35:00
64,2018-01-02 10:39:00,2685.0,5.50,1.75,5.20,1.60,2690.07,2690.07,2688.18,2688.76,...,4.10,7.25,2684.30,2690.9205,0.8505,2692.6215,2689.6448,True,09:35:00,2018-01-02 09:35:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
411702,2024-12-04 15:56:00,6065.0,21.50,0.05,21.50,0.05,6086.50,6086.68,6084.96,6085.69,...,8.45,18.65,6068.87,6086.5670,0.9159,6088.3988,6085.1932,True,10:01:00,2024-12-04 10:01:00
411703,2024-12-04 15:57:00,6065.0,0.00,0.05,0.00,0.05,6085.65,6086.61,6085.65,6085.82,...,8.45,18.65,6068.87,6086.6075,0.8596,6088.3267,6085.3181,True,10:01:00,2024-12-04 10:01:00
411704,2024-12-04 15:58:00,6065.0,22.61,0.00,22.61,0.00,6085.83,6087.05,6085.28,6087.05,...,8.45,18.65,6068.87,6086.7260,0.7341,6088.1942,6085.6249,True,10:01:00,2024-12-04 10:01:00
411705,2024-12-04 15:59:00,6065.0,21.40,0.00,21.40,0.00,6087.01,6087.02,6085.60,6086.22,...,8.45,18.65,6068.87,6086.7845,0.6332,6088.0509,6085.8347,True,10:01:00,2024-12-04 10:01:00


In [21]:
# Step 1: Create a condition where 'Close' is below 'LBB'
df_time['below_LBB'] = df_time['Close'] < df_time['Lower_Band']

# Step 2: Define a function to filter up to the first occurrence of the condition being true in each group
def filter_first_below_LBB(group):
    # Find the first index where the condition is met
    first_below_LBB = group[group['below_LBB']].index.min()
    
    # Filter the group up to that row (inclusive)
    return group.loc[:first_below_LBB]

# Step 3: Apply the function to each group
df_time = df_time.groupby(df_time['quote_datetime'].dt.date).apply(filter_first_below_LBB).reset_index(drop=True)

In [22]:
df_time_last = df_time[['quote_datetime']].groupby(df_time['quote_datetime'].dt.date).tail(1).reset_index(drop=True)

In [23]:
df_time_last

,quote_datetime
0,2018-01-02 10:39:00
1,2018-01-05 10:50:00
2,2018-01-08 10:42:00
3,2018-01-10 10:52:00
4,2018-01-17 15:21:00
...,...
1082,2024-11-26 11:13:00
1083,2024-11-27 11:02:00
1084,2024-12-02 10:51:00
1085,2024-12-03 10:46:00


In [24]:
df4.head()

,quote_datetime,strike,ask_C,ask_P,bid_C,bid_P,Open,High,Low,Close,...,put932,straddle932,spot932,Rolling_MA,Rolling_STD,Upper_Band,Lower_Band,call_below_10per,selling_time,first_quote_datetime
0,2018-01-02 09:35:00,2685.0,2.75,5.0,2.45,4.5,2683.80,2684.18,2683.05,2683.07,...,4.1,7.25,2684.3,2684.1000,0.9448,2685.9896,2682.6828,True,09:35:00,2018-01-02 09:35:00
1,2018-01-02 09:36:00,2685.0,2.65,5.0,2.55,4.7,2683.06,2683.23,2682.78,2682.94,...,4.1,7.25,2684.3,2683.8680,0.9688,2685.8056,2682.4148,True,09:35:00,2018-01-02 09:35:00
2,2018-01-02 09:37:00,2685.0,2.60,5.3,2.45,4.7,2682.94,2683.22,2682.36,2682.80,...,4.1,7.25,2684.3,2683.6900,0.9700,2685.6301,2682.2350,True,09:35:00,2018-01-02 09:35:00
3,2018-01-02 09:38:00,2685.0,3.30,4.3,3.10,3.8,2682.80,2684.01,2682.79,2683.90,...,4.1,7.25,2684.3,2683.7200,0.8891,2685.4981,2682.3864,True,09:35:00,2018-01-02 09:35:00
4,2018-01-02 09:39:00,2685.0,3.80,3.6,3.40,3.2,2683.84,2685.09,2683.84,2685.07,...,4.1,7.25,2684.3,2683.8888,0.9515,2685.7917,2682.4615,True,09:35:00,2018-01-02 09:35:00


In [25]:
# Step 1: Create a condition to filter rows in df4
df4_filtered = []

for date, last_quote_time in zip(df_time_last['quote_datetime'].dt.date, df_time_last['quote_datetime']):
    # Filter rows in df4 where 'quote_datetime' is before or equal to the last quote for that date
    filtered = df4[(df4['quote_datetime'].dt.date == date) & (df4['quote_datetime'] <= last_quote_time)]
    df4_filtered.append(filtered)

# Step 2: Combine all filtered parts into one DataFrame
df4_filtered = pd.concat(df4_filtered).reset_index(drop=True)

In [26]:
df4 = df4_filtered.copy()

In [27]:
# Ensure 'date' is a column or derived appropriately
df4['selling_call_price'] = (
    df4.groupby('date')['call_price']
    .transform('first')
)
df4['selling_call_price'] = df4['selling_call_price'].ffill()

In [28]:
# Ensure 'date' is a column or derived appropriately
df4['selling_put_price'] = (
    df4.groupby('date')['put_price']
    .transform('first')
)
df4['selling_put_price'] = df4['selling_put_price'].ffill()

In [29]:
df4.head()

,quote_datetime,strike,ask_C,ask_P,bid_C,bid_P,Open,High,Low,Close,...,spot932,Rolling_MA,Rolling_STD,Upper_Band,Lower_Band,call_below_10per,selling_time,first_quote_datetime,selling_call_price,selling_put_price
0,2018-01-02 09:35:00,2685.0,2.75,5.0,2.45,4.5,2683.80,2684.18,2683.05,2683.07,...,2684.3,2684.1000,0.9448,2685.9896,2682.6828,True,09:35:00,2018-01-02 09:35:00,2.6,4.75
1,2018-01-02 09:36:00,2685.0,2.65,5.0,2.55,4.7,2683.06,2683.23,2682.78,2682.94,...,2684.3,2683.8680,0.9688,2685.8056,2682.4148,True,09:35:00,2018-01-02 09:35:00,2.6,4.75
2,2018-01-02 09:37:00,2685.0,2.60,5.3,2.45,4.7,2682.94,2683.22,2682.36,2682.80,...,2684.3,2683.6900,0.9700,2685.6301,2682.2350,True,09:35:00,2018-01-02 09:35:00,2.6,4.75
3,2018-01-02 09:38:00,2685.0,3.30,4.3,3.10,3.8,2682.80,2684.01,2682.79,2683.90,...,2684.3,2683.7200,0.8891,2685.4981,2682.3864,True,09:35:00,2018-01-02 09:35:00,2.6,4.75
4,2018-01-02 09:39:00,2685.0,3.80,3.6,3.40,3.2,2683.84,2685.09,2683.84,2685.07,...,2684.3,2683.8888,0.9515,2685.7917,2682.4615,True,09:35:00,2018-01-02 09:35:00,2.6,4.75


In [30]:
# Calculate the number of unique days
num_unique_days = df4['quote_datetime'].dt.date.nunique()

# Display the result
print(num_unique_days)

1087


In [31]:
# df4_notna_call_above = df4_notna[df4_notna['call_below_1st'] != True]

In [32]:
# # Ensure 'date' is a column or derived appropriately
# df4_notna_call_below['call_sell_again_price'] = (
#     df4_notna_call_below.groupby('date')['call_price']
#     .transform('first')
# )
# df4_notna_call_below['call_sell_again_price'] = df4_notna_call_below['call_sell_again_price'].ffill()

In [33]:
# def exit_strategy2(df):
#     filtered_df_list = []
    
#     # Group by each date
#     for date, group in df.groupby('date'):
#         # Step 1: Get the first 'call_pnl' value for the date (first_straddle)
#         first_call = group['call_price'].iloc[0]
        
#         # Step 2: Define the stop-loss level
#         stop_loss_level = 1.5 * first_call
        
#         # Step 3: Apply the stop-loss logic
#         stop_loss_triggered = False  # Track if the stop-loss condition has been hit
#         #condition_triggered = False
#         group['call_stop2_price'] = np.nan
#         group['stoploss_time2'] = np.nan
        
#         for i in range(len(group)):
#             if not stop_loss_triggered and group['call_price'].iloc[i] >= stop_loss_level:
#                 stop_loss_triggered = True
#                 a = i
#                 #stop_loss_time = group['time'].iloc[i] 
            
#             # Forward-fill the stop-loss level if triggered
#             if stop_loss_triggered:
#                 group['call_stop2_price'] = stop_loss_level
#                 #group['stoploss1'] = True
#                 group['stoploss_time2'] = group.loc[group.index[a], 'time']
#                 #group['stoploss_time'] = stop_loss_time
        
#         # Step 4: Add the adjusted group to the filtered list
#         filtered_df_list.append(group)
    
#     # Combine all the filtered groups
#     return pd.concat(filtered_df_list, ignore_index=True)

In [34]:
df_trade = df4[df4['call_below_10per'] == True]

In [35]:
# Calculate the number of unique days
num_unique_days = df_trade['quote_datetime'].dt.date.nunique()

# Display the result
print(num_unique_days)

1087


In [36]:
# def exit_strategy(df):
#     filtered_df_list = []
    
#     # Group by each date
#     for date, group in df.groupby('date'):
#         # Step 1: Get the first 'Close' value for each date (first_close)
#         #first_close = group['Close'].iloc[0]
#         first_call = group['call_price'].iloc[0]
        
#         # Step 2: Calculate the condition and filter the rows accordingly
#         condition = group['call_price'] > 0.5 * first_call #+ first_close
        
#         # Step 3: Find the first row where the condition fails and filter accordingly
#         if condition.any():
#             first_fail_idx = condition[~condition].index[0] if (~condition).any() else None
            
#             # Keep rows before the first failure (inclusive of the row where it fails)
#             if first_fail_idx:
#                 filtered_group = group.loc[:first_fail_idx]
#             else:
#                 filtered_group = group  # Keep the entire group if the condition never fails
            
#             filtered_df_list.append(filtered_group)
    
#     return pd.concat(filtered_df_list, ignore_index=True)

In [37]:
# df_trade = exit_strategy(df_trade)

In [38]:
# Ensure the DataFrame is sorted by 'date' and 'time'
df_trade = df_trade.sort_values(by=['date', 'time'])
df_trade['call_decay_30per'] = False
df_trade['decay30time'] = np.nan

# Define a function to filter each group based on the first occurrence of a condition
def filter_first_true1(group):
    # Find the index of the first occurrence where 'call_price' <= 0.7 * 'selling_call_price'
    first_yes_idx = group[group['call_price'] <= 0.7 * group['selling_call_price']].index.min()
    
    # If such an index exists, update 'decay30time' and 'call_decay_30per' columns
    if not pd.isna(first_yes_idx):
        group['call_decay_30per'] = True
        group['decay30time'] = group.loc[first_yes_idx, 'time']
    
    return group

# Apply the filtering function to each group based on 'date'
df_trade = df_trade.groupby('date').apply(filter_first_true1).reset_index(drop=True)

In [39]:
# df_trade

In [40]:
def exit_strategy2(df):
    filtered_df_list = []
    
    # Group by each date
    for date, group in df.groupby('date'):
        # Step 1: Get the first 'call_pnl' value for the date (first_straddle)
        first_call = group['call_price'].iloc[0]
        
        # Step 2: Define the stop-loss level
        stop_loss_level = 1.5 * first_call
        
        # Step 3: Apply the stop-loss logic
        stop_loss_triggered = False  # Track if the stop-loss condition has been hit
        #condition_triggered = False
        group['call_stop1_price'] = np.nan
        group['stoploss_time'] = np.nan
        group['put_price_call_stop'] = np.nan
        
        for i in range(len(group)):
            if not stop_loss_triggered and group['call_price'].iloc[i] >= stop_loss_level:
                stop_loss_triggered = True
                a = i
                #stop_loss_time = group['time'].iloc[i] 
            
            # Forward-fill the stop-loss level if triggered
            if stop_loss_triggered:
                group['call_stop1_price'] = group.loc[group.index[a], 'call_price']
                #group['stoploss1'] = True
                group['stoploss_time'] = group.loc[group.index[a], 'time']
                group['put_price_call_stop'] = group.loc[group.index[a], 'put_price']
                #group['stoploss_time'] = stop_loss_time
        
        # Step 4: Add the adjusted group to the filtered list
        filtered_df_list.append(group)
    
    # Combine all the filtered groups
    return pd.concat(filtered_df_list, ignore_index=True)

In [41]:
df_trade1 = exit_strategy2(df_trade)

In [42]:
# Calculate the number of unique days
num_unique_days = df_trade1['quote_datetime'].dt.date.nunique()

# Display the result
print(num_unique_days)

1087


In [43]:
df_trade11 = df_trade1[
    df_trade1['decay30time'].notna() & 
    (df_trade1['stoploss_time'].isna() | (df_trade1['stoploss_time'] > df_trade1['decay30time']))
]

In [44]:
# Identify the unique dates in df_trade11
excluded_dates = df_trade11['date'].unique()

# Filter df_trade1 to exclude rows with the dates in excluded_dates
df_trade_filtered = df_trade1[~df_trade1['date'].isin(excluded_dates)]

In [45]:
# Calculate the number of unique days
num_unique_days = df_trade11['quote_datetime'].dt.date.nunique()

# Display the result
print(num_unique_days)

636


In [46]:
# Calculate the number of unique days
num_unique_days = df_trade_filtered['quote_datetime'].dt.date.nunique()

# Display the result
print(num_unique_days)

451


In [47]:
def exit_strategy3(df):
    filtered_df_list = []
    
    # Group by each date
    for date, group in df.groupby('date'):
        # Step 1: Get the first 'call_price' value for the date
        first_call = group['call_price'].iloc[0]
        
        # Step 2: Define the stop-loss level
        stop_loss_level = 1 * first_call
        
        # Step 3: Initialize decay_time and other variables
        decay_time = group['decay30time'].iloc[0] 
        stop_loss_triggered = False
        group['call_stop1_price'] = np.nan
        group['stoploss_time'] = np.nan
        group['put_price_call_stop'] = np.nan
        
        for i in range(len(group)):
            current_time = group['time'].iloc[i]
            current_price = group['call_price'].iloc[i]
            
            # Check for stop-loss condition including decay time
            if (not stop_loss_triggered and 
                current_price >= stop_loss_level and 
                current_time >= decay_time):
                stop_loss_triggered = True
                stop_loss_index = i
            
            # Forward-fill the stop-loss level if triggered
            if stop_loss_triggered:
                group['call_stop1_price'] = group.loc[group.index[stop_loss_index], 'call_price']
                group['stoploss_time'] = group.loc[group.index[stop_loss_index], 'time']
                group['put_price_call_stop'] = group.loc[group.index[stop_loss_index], 'put_price']
        
        # Step 4: Add the adjusted group to the filtered list
        filtered_df_list.append(group)
    
    # Combine all the filtered groups
    return pd.concat(filtered_df_list, ignore_index=True)

In [48]:
df_trade111 = exit_strategy3(df_trade11)

In [49]:
df_trade11

,quote_datetime,strike,ask_C,ask_P,bid_C,bid_P,Open,High,Low,Close,...,call_below_10per,selling_time,first_quote_datetime,selling_call_price,selling_put_price,call_decay_30per,decay30time,call_stop1_price,stoploss_time,put_price_call_stop
65,2018-01-05 09:34:00,2730.0,4.30,2.65,3.90,2.45,2732.89,2733.07,2732.08,2732.27,...,True,09:34:00,2018-01-05 09:34:00,4.10,2.55,True,09:38:00,NaN,NaN,NaN
66,2018-01-05 09:35:00,2730.0,3.50,3.40,3.20,3.20,2732.20,2732.20,2730.19,2730.48,...,True,09:34:00,2018-01-05 09:34:00,4.10,2.55,True,09:38:00,NaN,NaN,NaN
67,2018-01-05 09:36:00,2730.0,3.00,3.70,2.95,3.20,2730.46,2730.46,2729.38,2729.72,...,True,09:34:00,2018-01-05 09:34:00,4.10,2.55,True,09:38:00,NaN,NaN,NaN
68,2018-01-05 09:37:00,2730.0,3.40,3.60,3.20,3.30,2729.75,2730.34,2729.53,2730.17,...,True,09:34:00,2018-01-05 09:34:00,4.10,2.55,True,09:38:00,NaN,NaN,NaN
69,2018-01-05 09:38:00,2730.0,2.90,3.90,2.75,3.60,2730.15,2730.15,2729.29,2729.33,...,True,09:34:00,2018-01-05 09:34:00,4.10,2.55,True,09:38:00,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88184,2024-12-04 11:27:00,6065.0,0.00,3.22,0.00,3.22,6074.62,6075.20,6074.14,6074.94,...,True,10:01:00,2024-12-04 10:01:00,8.59,7.90,True,10:16:00,12.95,11:19:00,3.15
88185,2024-12-04 11:28:00,6065.0,12.25,3.30,12.25,3.30,6074.97,6075.09,6074.11,6074.34,...,True,10:01:00,2024-12-04 10:01:00,8.59,7.90,True,10:16:00,12.95,11:19:00,3.15
88186,2024-12-04 11:29:00,6065.0,12.30,3.20,12.30,3.20,6074.38,6074.62,6074.15,6074.58,...,True,10:01:00,2024-12-04 10:01:00,8.59,7.90,True,10:16:00,12.95,11:19:00,3.15
88187,2024-12-04 11:30:00,6065.0,0.00,3.25,0.00,3.25,6074.65,6074.96,6074.29,6074.31,...,True,10:01:00,2024-12-04 10:01:00,8.59,7.90,True,10:16:00,12.95,11:19:00,3.15


In [50]:
df_trade111

,quote_datetime,strike,ask_C,ask_P,bid_C,bid_P,Open,High,Low,Close,...,call_below_10per,selling_time,first_quote_datetime,selling_call_price,selling_put_price,call_decay_30per,decay30time,call_stop1_price,stoploss_time,put_price_call_stop
0,2018-01-05 09:34:00,2730.0,4.30,2.65,3.90,2.45,2732.89,2733.07,2732.08,2732.27,...,True,09:34:00,2018-01-05 09:34:00,4.10,2.55,True,09:38:00,4.45,10:09:00,1.625
1,2018-01-05 09:35:00,2730.0,3.50,3.40,3.20,3.20,2732.20,2732.20,2730.19,2730.48,...,True,09:34:00,2018-01-05 09:34:00,4.10,2.55,True,09:38:00,4.45,10:09:00,1.625
2,2018-01-05 09:36:00,2730.0,3.00,3.70,2.95,3.20,2730.46,2730.46,2729.38,2729.72,...,True,09:34:00,2018-01-05 09:34:00,4.10,2.55,True,09:38:00,4.45,10:09:00,1.625
3,2018-01-05 09:37:00,2730.0,3.40,3.60,3.20,3.30,2729.75,2730.34,2729.53,2730.17,...,True,09:34:00,2018-01-05 09:34:00,4.10,2.55,True,09:38:00,4.45,10:09:00,1.625
4,2018-01-05 09:38:00,2730.0,2.90,3.90,2.75,3.60,2730.15,2730.15,2729.29,2729.33,...,True,09:34:00,2018-01-05 09:34:00,4.10,2.55,True,09:38:00,4.45,10:09:00,1.625
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51170,2024-12-04 11:27:00,6065.0,0.00,3.22,0.00,3.22,6074.62,6075.20,6074.14,6074.94,...,True,10:01:00,2024-12-04 10:01:00,8.59,7.90,True,10:16:00,8.90,10:26:00,6.420
51171,2024-12-04 11:28:00,6065.0,12.25,3.30,12.25,3.30,6074.97,6075.09,6074.11,6074.34,...,True,10:01:00,2024-12-04 10:01:00,8.59,7.90,True,10:16:00,8.90,10:26:00,6.420
51172,2024-12-04 11:29:00,6065.0,12.30,3.20,12.30,3.20,6074.38,6074.62,6074.15,6074.58,...,True,10:01:00,2024-12-04 10:01:00,8.59,7.90,True,10:16:00,8.90,10:26:00,6.420
51173,2024-12-04 11:30:00,6065.0,0.00,3.25,0.00,3.25,6074.65,6074.96,6074.29,6074.31,...,True,10:01:00,2024-12-04 10:01:00,8.59,7.90,True,10:16:00,8.90,10:26:00,6.420


In [51]:
# Assuming df1 and df2 are your DataFrames
combined_df = pd.concat([df_trade_filtered, df_trade111], ignore_index=True)

# Sort the combined DataFrame by 'quote_datetime'
combined_df_sorted = combined_df.sort_values(by='quote_datetime').reset_index(drop=True)

In [52]:
# Calculate the number of unique days
num_unique_days = combined_df_sorted['quote_datetime'].dt.date.nunique()

# Display the result
print(num_unique_days)

1087


In [53]:
df_trade3 = combined_df_sorted.groupby('date').tail(1).reset_index(drop=True)

In [54]:
df_trade3['call_pnl'] = df_trade3.apply(
    lambda row: (
        row['selling_call_price'] - row['call_price']
    ) if pd.isna(row['call_stop1_price']) else (
        row['selling_call_price'] - row['call_stop1_price']
    ),
    axis=1
)

In [55]:
df_trade3['put_pnl'] = df_trade3.apply(
    lambda row: (
        row['selling_put_price'] - row['put_price']
    ) if pd.isna(row['put_price_call_stop']) else (
        row['selling_put_price'] - row['put_price_call_stop']
    ),
    axis=1
)

In [56]:
# df_trade3['put_pnl'] *=-1

In [57]:
# Calculate the number of unique days
num_unique_days = df_trade3['quote_datetime'].dt.date.nunique()

# Display the result
print(num_unique_days)

1087


In [58]:
df_trade3 = df_trade3.sort_values(by='quote_datetime').reset_index(drop=True)

In [59]:
df_trade3 = df_trade3[df_trade3['call_price'] >= 0]

In [60]:
# df_filtered1['call932'] = df_filtered1['call_price'].where(df_filtered1['quote_datetime'].dt.time == pd.to_datetime('09:32:00').time())
# df_filtered1['call932'] = df_filtered1['call932'].ffill()    

# df_filtered1['call_pnl'] = df_filtered1['call932'] - df_filtered1['call_price']

In [61]:
# df_filtered1['put_price'] /= 2
# df_filtered1['put932'] = df_filtered1['put_price'].where(df_filtered1['quote_datetime'].dt.time == pd.to_datetime('09:32:00').time())
# df_filtered1['put932'] = df_filtered1['put932'].ffill()    

# df_filtered1['put_pnl'] = df_filtered1['put932'] - df_filtered1['put_price']

In [62]:
# df_filtered1['time'] = df_filtered1['quote_datetime'].dt.time

In [63]:
# df_filtered1['straddle_pnl_bps'] =  (df_filtered1['straddle_pnl']/ df_filtered1['spot932'])*10000
df_trade3['put_pnl_bps'] =  (df_trade3['put_pnl']/ df_trade3['spot932'])*10000
df_trade3['call_pnl_bps'] =  (df_trade3['call_pnl']/ df_trade3['spot932'])*10000

In [64]:
df5 = df_trade3[['quote_datetime', 'date', 'time', 'strike932', 'low931', 'high931', 'open931', 'close931', 'Close', 'spot932', 'gap931', 'realized_volatility_post30min', 'prev_close', 'gap', 'prev_day_neg', 'range', 'realized_volatility_full_day', 'sma_50', 'sma_21', 'intraday_sma_21', 'below_sma_50', 'day_close', 'straddle932', 'call932', 'call_stop1_price', 'selling_call_price', 'call_price', 'call_pnl', 'call_pnl_bps', 'selling_put_price', 'put932', 'put_price', 'put_price_call_stop', 'put_pnl', 'put_pnl_bps', 'selling_time', 'stoploss_time']]

In [65]:
# df1['straddleinpercent'] = (df1['straddle932']/df1['spot932'])*100

In [66]:
# Calculate the number of unique days
num_unique_days = df5['quote_datetime'].dt.date.nunique()

# Display the result
print(num_unique_days)

1087


In [67]:
# df5 = df_trade6.copy()

In [68]:
df5['year'] = df5['quote_datetime'].dt.year

In [69]:
df5 = df5[df5['prev_day_neg'] == 'yes']

In [70]:
df5 = df5[df5['below_sma_50'] == 'No']

In [71]:
df5 = df5[df5['realized_volatility_full_day'] > 0.1]

In [72]:
df5 = df5[df5['gap']/df5['straddle932'] < 1]

In [73]:
# df5 = df5[df5['gap']/df5['straddle932'] < 0.5]

In [74]:
# Calculate the number of unique days
num_unique_days = df5['quote_datetime'].dt.date.nunique()

# Display the result
print(num_unique_days)

103


In [151]:
# df5.to_csv('put.csv')

In [153]:
df5['straddlepercent'] = (df5['straddle932']/df5['strike932'])*100

In [155]:
df5['vol_adjusted'] = np.where(
    df5['straddlepercent'] < 0.5,  # 0.5% as a decimal
    df5['call_pnl_bps'], 
    df5['call_pnl_bps'] * (0.5 / df5['straddlepercent'])  # Adjusted value
)

In [157]:
df5.head()

,quote_datetime,date,time,strike932,low931,high931,open931,close931,Close,spot932,...,put932,put_price,put_price_call_stop,put_pnl,put_pnl_bps,selling_time,stoploss_time,year,straddlepercent,vol_adjusted
12,2018-02-09 10:36:00,2018-02-09,10:36:00,2610.0,2601.78,2608.19,2601.78,2606.40,2601.49,2613.07,...,14.15,18.35,NaN,-1.85,-7.0798,09:35:00,NaN,2018,1.2261,10.6125
19,2018-02-28 10:43:00,2018-02-28,10:43:00,2755.0,2753.78,2757.17,2753.78,2756.86,2757.11,2756.73,...,6.85,6.20,NaN,1.65,5.9854,09:35:00,NaN,2018,0.5590,-1.2979
20,2018-03-02 11:00:00,2018-03-02,11:00:00,2660.0,2655.62,2658.89,2658.89,2657.73,2659.62,2660.51,...,13.25,11.60,13.05,1.90,7.1415,09:37:00,10:02:00,2018,1.0395,-0.0904
23,2018-03-14 11:03:00,2018-03-14,11:03:00,2775.0,2773.92,2776.56,2774.06,2776.51,2762.89,2775.12,...,6.20,13.50,6.70,1.15,4.1440,09:34:00,10:02:00,2018,0.4324,-0.1802
36,2018-04-20 10:44:00,2018-04-20,10:44:00,2695.0,2692.56,2693.80,2692.56,2693.76,2683.80,2693.56,...,6.40,12.35,NaN,-4.90,-18.1915,09:34:00,NaN,2018,0.4082,9.5598


In [159]:
# Initialize an empty DataFrame to store yearly drawdown information
yearly_drawdowns = []

# Group the DataFrame by year and calculate the drawdowns
for year, group in df5.groupby('year'):
    # Calculate the cumulative sum of pnl_bps
    group['cumulative_pnl'] = group['vol_adjusted'].cumsum()
    
    # Calculate the running maximum of the cumulative pnl
    group['running_max'] = group['cumulative_pnl'].cummax()
    
    # Calculate the drawdown as the difference between the running max and the current cumulative pnl
    group['drawdown'] = group['running_max'] - group['cumulative_pnl']
    
    # Find the maximum drawdown for the year
    max_drawdown = group['drawdown'].max()
    
    # Append the results to the list
    yearly_drawdowns.append({'year': year, 'max_drawdown': max_drawdown})

# Create a DataFrame from the yearly drawdown information
yearly_drawdowns_df = pd.DataFrame(yearly_drawdowns)

print(yearly_drawdowns_df)

   year  max_drawdown
0  2018       24.3774
1  2019       13.9064
2  2020       16.9335
3  2021       22.0415
4  2022       26.4979
5  2023       13.5213
6  2024       22.2886


In [161]:
df5.groupby('year')['vol_adjusted'].sum().reset_index()

,year,vol_adjusted
0,2018,11.9812
1,2019,63.0305
2,2020,54.4314
3,2021,42.7119
4,2022,102.6958
5,2023,53.0332
6,2024,20.1271


In [163]:
df5.to_excel('marker1_1032_rentry50_all1.xlsx')

In [82]:
df5['call_pnl_bps'].sum()

626.398117842722